In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
       # print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!nvidia-smi

In [ ]:
import fastai
from fastai.vision.all import *
from fastai.vision.core import *
from fastai.data.all import *

from sklearn.metrics import *
from sklearn.model_selection import train_test_split

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
BASE_PATH = "../input/medical-masks-part1/images/"

In [ ]:
df = pd.read_csv('../input/medical-masks-part1/df.csv')

In [ ]:
df['name'] = BASE_PATH + df['name']

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
#Error file is:
#../input/medical-masks-part1/images/000030_1_000030_NONE_29.jpg at index = 116
#../input/medical-masks-part1/images/007790_1_005591_NONE_27.jpg at index = 31156

In [ ]:
corrupt = []

for i, row in df.iterrows():
    try:
        image = Image.open(row['name'])
    except:
        corrupt.append(i)
        print(str(i) + row['name'])
        
df.drop(corrupt, inplace=True)

In [ ]:
df.drop([116,31156], inplace=True)

In [ ]:
train, test = train_test_split(df, stratify=df['TYPE'], test_size=0.2)

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
fm =  DataBlock(blocks = (ImageBlock, CategoryBlock),
                 splitter=RandomSplitter(valid_pct=0.2),
                 get_x=ColReader(5),
                 get_y=ColReader(1),
                 item_tfms=RandomResizedCrop(224),
                 batch_tfms=[Normalize.from_stats(*imagenet_stats)])

In [ ]:
dls = fm.dataloaders(train, batch_size=16)

In [ ]:
dls_test = fm.dataloaders(test, batch_size=16)

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=[error_rate,accuracy])

In [ ]:
# copy model weights to working directory
!cp ../input/facemask-weights/2epoch.pth ./models

In [ ]:
# load model (if trained)
learn = learn.load('2epoch')

In [ ]:
# learning rate finder
lr_min, lr_steep, lr_valley, lr_slide = learn.lr_find(suggest_funcs=(minimum, steep, valley, slide))

In [ ]:
# train model
# skip if trained
learn.fine_tune(2, base_lr=lr_valley)

In [ ]:
interp_train = ClassificationInterpretation.from_learner(learn)
interp_train.plot_confusion_matrix()

In [ ]:
learn.dls = dls_test

In [ ]:
interp_test = ClassificationInterpretation.from_learner(learn)
interp_test.plot_confusion_matrix()

In [ ]:
# learn.save('2epoch') ==> equivalent to code below

In [ ]:
# torch.save(learn.model.state_dict(), 'model.pth') ==> save weight state

In [ ]:
torch.save(learn.model, 'model.pth')